In [1]:
import pandas as pd
import numpy as np

In [6]:
d1 = {'Social_media_followers':[1000000, np.nan, 2000000, 1310000, 1700000, np.nan, 4100000, 22000000, 100000, 2000000],
      'Sold_out':[1, 0, 0, 1, 0, 0,0, 1, 0, 1]}

In [7]:
df1 = pd.DataFrame(data=d1)

In [8]:
df1

,Social_media_followers,Sold_out
0,1000000.0,1
1,NaN,0
2,2000000.0,0
3,1310000.0,1
4,1700000.0,0
5,NaN,0
6,4100000.0,0
7,22000000.0,1
8,100000.0,0
9,2000000.0,1


In [9]:
X1 = df1[['Social_media_followers']]

In [10]:
y1 = df1[['Sold_out']]

In [11]:
from sklearn.model_selection import train_test_split

In [13]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.3, random_state= 19 )

In [14]:
X1_train.shape

(7, 1)

In [15]:
X1_test.shape

(3, 1)

In [16]:
from sklearn.impute import SimpleImputer

In [17]:
imputer = SimpleImputer(strategy='mean')

In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
lr = LogisticRegression()

In [20]:
from sklearn.pipeline import make_pipeline

In [22]:
pipe1 = make_pipeline(imputer, lr)

In [23]:
pipe1.fit(X1_train, y1_train)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('logisticregression', LogisticRegression())])

In [24]:
pipe1.score(X1_train, y1_train)

0.5714285714285714

In [25]:
pipe1.score(X1_test, y1_test)

0.3333333333333333

In [27]:
pipe1.named_steps.simpleimputer.statistics_

array([1701666.66666667])

In [28]:
pipe1.named_steps.logisticregression.coef_

array([[-8.0563611e-07]])

In [29]:
#more advanced pipeline

In [30]:
d2 = {'Genre':['Rock', 'Metal', 'Bluegrass', 'Rock', np.nan, 'Rock', 'Rock', np.nan, 'Bluegrass', 'Rock'],
    'Social_media_followers':[1000000, np.nan, 2000000, 1310000, 1700000, np.nan, 4100000, 1600000, 2200000, 1000000],
    'Sold_out':[1,0,0,1,0,0,0,1,0,1]}

In [31]:
df = pd.DataFrame(data=d2)

In [32]:
df


,Genre,Social_media_followers,Sold_out
0,Rock,1000000.0,1
1,Metal,NaN,0
2,Bluegrass,2000000.0,0
3,Rock,1310000.0,1
4,NaN,1700000.0,0
5,Rock,NaN,0
6,Rock,4100000.0,0
7,NaN,1600000.0,1
8,Bluegrass,2200000.0,0
9,Rock,1000000.0,1


In [33]:
X = df.iloc[:, 0:2]

In [34]:
y= df.iloc[:,2]

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state= 75)

In [36]:
num_cols = ['Social_media_followers']

In [37]:
cat_cols = ['Genre']

In [38]:
from sklearn.pipeline import Pipeline

In [39]:
from sklearn.preprocessing import StandardScaler

In [41]:
num_pipeline = Pipeline(steps =[
    ('impute', SimpleImputer(strategy = 'mean')),
    ('scale', StandardScaler() )
]

)

In [42]:
from sklearn.preprocessing import OneHotEncoder

In [46]:
cat_pipeline = Pipeline(steps = [
    ('impute', SimpleImputer(strategy = 'most_frequent')), #preenche valores ausentes
    ('one-hot-encoder', OneHotEncoder(handle_unknown='ignore', sparse_output = False))


])

In [47]:
from sklearn.compose import ColumnTransformer

In [49]:
col_transformer = ColumnTransformer(transformers = [
    ('num_pipeline', num_pipeline, num_cols),
    ('cat_pipeline', cat_pipeline, cat_cols),

],
    remainder = 'drop',
    n_jobs = -1
)

This code defines a ColumnTransformer object named col_transformer. This object is a crucial part of  data preprocessing in ml.
this is a class module used to apply different transformations to different columns of a dataset.

- transformers: this argument is the heart of ColumnTransformer. It takes a list of tuples, where each tuple defines a transformation to be applied.

- The columnTransformer acts like a smart tool that applies specific cleaning and preparation steps to each column type.

In [51]:
from sklearn.tree import DecisionTreeClassifier # Corrected the typo in the class name

In [52]:
dtc = DecisionTreeClassifier()

In [53]:
pipefinal = make_pipeline(col_transformer, dtc)

In [54]:
pipefinal.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   StandardScaler())]),
                                                  ['Social_media_followers']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot-encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Genre'])])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [55]:
pipefinal.score(X_test, y_test)

0.6666666666666666

In [56]:
#how to save your pipeline
import joblib

In [57]:
joblib.dump(pipefinal, "pipe.joblib")

['pipe.joblib']

In [58]:
pipefinal2= joblib.load("pipe.joblib")